Sample use of the algorithm with Jupyter.

# Notebook initialization
## Download module

In [ ]:
import os
import urllib.request
import importlib
import matplotlib.pyplot as plt
plt.close('all')

# if not os.path.isfile('mtf_estimator_algorithm.py'):
#     urllib.request.urlretrieve('https://raw.githubusercontent.com/JorgeGIlG/MTF_Estimator/devel/qgis_plugin/mtf_estimator/mtf_estimator_algorithm.py',
#                                'mtf_estimator_algorithm.py')


## Download sample image

In [ ]:
# if not os.path.isfile('baotou_CALVAL_L0R_000000_20200328T032332_202003T032334_MTF_12196_7333.tif'):
#     urllib.request.urlretrieve('https://github.com/JorgeGIlG/MTF_Estimator/raw/devel/data/baotou_CALVAL_L0R_000000_20200328T032332_202003T032334_MTF_12196_7333.tif',
#                                'baotou_CALVAL_L0R_000000_20200328T032332_202003T032334_MTF_12196_7333.tif')


In [ ]:
import mtf_estimator_algorithm
import osgeo
import numpy as np
from osgeo import ogr, osr, gdal
importlib.reload(mtf_estimator_algorithm)

%matplotlib widget


# Declare inputs

In [ ]:


# Raster to mask and crop using Aoi_Wkt
Image_File = 'image-for-psf-0-loc1.png'
# Band number. First is 1
Band_n = 1


# Read, mask and crop input file

In [ ]:
def process(Aoi_Wkt):
    gdal_layer = gdal.Open(Image_File, gdal.GA_ReadOnly)
    gt = list(gdal_layer.GetGeoTransform())
    xsize = gdal_layer.RasterXSize
    ysize = gdal_layer.RasterYSize
    band = gdal_layer.GetRasterBand(Band_n)
    raster_srs = osr.SpatialReference()
    raster_srs.ImportFromWkt(gdal_layer.GetProjection())
    vector_srs = raster_srs


    # https://gdal.org/tutorials/osr_api_tut.html#crs-and-axis-order
    if int(osgeo.__version__[0]) >= 3:
        # GDAL 3 changes axis order: https://github.com/OSGeo/gdal/issues/1546
        raster_srs.SetAxisMappingStrategy(osgeo.osr.OAMS_TRADITIONAL_GIS_ORDER)
        vector_srs.SetAxisMappingStrategy(osgeo.osr.OAMS_TRADITIONAL_GIS_ORDER)

    if str(raster_srs) == '':
        coord_transform = None
        # self.console('WARNING: Raster with no CRS')
        gt[5] = -1*gt[5]
    else:
        coord_transform = osr.CoordinateTransformation(vector_srs, raster_srs)


    memlayer_drv = ogr.GetDriverByName('Memory')
    memlayer_ds = memlayer_drv.CreateDataSource('')
    memlayer = memlayer_ds.CreateLayer('aoi', raster_srs, geom_type=ogr.wkbPolygon)
    memlayer.CreateField(ogr.FieldDefn('id', ogr.OFTInteger))
    featureDefn = memlayer.GetLayerDefn()
    memfeat = ogr.Feature(featureDefn)
    geom = ogr.CreateGeometryFromWkt(Aoi_Wkt)
    if not coord_transform is None:
        geom.Transform(coord_transform)

    memfeat.SetGeometry(geom)
    memlayer.CreateFeature(memfeat)

    # Get extent in raster coords
    e = np.array(memlayer.GetExtent()).copy()
    e = np.reshape(e, [2, 2])
    e = np.array(np.meshgrid(e[0], e[1]))
    E = e.T.reshape(-1, 2)
    m = np.reshape(np.array(gt).copy(), [2, 3])
    A = m[:, 0]
    m = m[:, 1:]
    M = np.linalg.inv(m)
    col_list, row_list = np.matmul(M, (E-A).T)
    pxoffset = 5
    col_min = int(np.max([np.floor(np.min(col_list)) - pxoffset, 1]))
    col_max = int(np.min([np.ceil(np.max(col_list))+pxoffset, xsize-1]))
    row_min = int(np.max([np.floor(np.min(row_list)) - pxoffset, 1]))
    row_max = int(np.min([np.ceil(np.max(row_list))+pxoffset, ysize-1]))
    sub_gt = gt
    sub_gt[0] = gt[0] + gt[1]*col_min + gt[2]*row_min
    sub_gt[3] = gt[3] + gt[4]*col_min + gt[5]*row_min
    sub_xsize = int(col_max-col_min)
    sub_ysize = int(row_max-row_min)

    memraster_drv = gdal.GetDriverByName('MEM')
    print(sub_xsize, sub_ysize)
    memraster = memraster_drv.Create('', sub_xsize, sub_ysize, 1, band.DataType)

    memraster.SetProjection(gdal_layer.GetProjection())
    memraster.SetGeoTransform(sub_gt)
    memband = memraster.GetRasterBand(1)
    memband.WriteArray(np.zeros([sub_ysize, sub_xsize]))
    gdal.RasterizeLayer(memraster, [1], memlayer, burn_values=[1])
    mask = memband.ReadAsArray(0, 0, sub_xsize, sub_ysize)
    memband.WriteArray(mask*band.ReadAsArray(col_min, row_min, sub_xsize, sub_ysize))
    mask = None
    return memraster


# Run algorithm

In [ ]:
importlib.reload(mtf_estimator_algorithm)
# Area of Interest polygon in WKT
# Aoi_Wkt = '''
# Polygon ((
# 584 -175,
# 630 -184,
# 593 -297,
# 549 -289,
# 584 -175
# ))
# '''

# Aoi_Wkt = '''
# Polygon ((
# 100 -10,
# 632 -10,
# 632 -445,
# 100 -445,
# 100 -10
# ))
# '''


# Aoi_Wkt = '''
# Polygon ((586.67413213885777168 -173.40201567749164724, 553.65733482642781382 -288.25531914893622343, 584.13437849944011759 -299.82530795072790397, 617.9977603583427026 -181.58566629339310339, 586.67413213885777168 -173.40201567749164724))'''

# Aoi_Wkt = '''
# Polygon ((577.88654060066733109 -174.47719688542827043, 632.26696329254718876 -188.49276974416017083, 597.50834260289207123 -304.82202447163518855, 538.92324805339262639 -280.99555061179091808, 577.88654060066733109 -174.47719688542827043))
# '''

Aoi_Wkt = '''Polygon ((589.09899888765289688 -177.28031145717466188, 617.13014460511681136 -185.40934371523917434, 603.95550611790872608 -220.16796440489434872, 580.12903225806451246 -212.31924360400444129, 589.09899888765289688 -177.28031145717466188))'''

# Aoi_Wkt = '''Polygon ((572.2803114571745482 -174.19688542825360855, 634.78976640711903201 -190.73526140155729536, 603.11457174638485412 -300.33704115684093949, 538.08231368186875443 -280.15461624026698928, 572.2803114571745482 -174.19688542825360855))'''


mtf_estimator_algorithm.Mtf(process(Aoi_Wkt))


In [ ]:
# def sigmoid(x, a, b, l, s):
#     return a+b*(1/(1+np.power(np.e, -l*(x+s))))

def test(L=None):
    import matplotlib.pyplot as plt
    x = [-9.95768915383951, 9.97456435677212]
    OverSampFreq = np.float64(1e3)   
    a, b, l, s, optSmooth = 0.009548118522661594, 0.9765408418125651, 2.883940582009699, 0.004689178501557088, 0.13395440581238172
    if L is not None:
        l = L
    xAux = np.arange(np.min(x), np.max(x), step=1/OverSampFreq)    
    S = mtf_estimator_algorithm.sigmoid(xAux, a, b, l, s)
    fig, ax = plt.subplots(1,1)
    ax.plot(xAux, S, '.')
   
    def get_position(data, value):
        return np.argmin(np.abs(data-value))



    center_pos = get_position(S, s)    
    max_pos = get_position(S[center_pos:], np.quantile(S[center_pos:], 0.75))
    distance_pos = max_pos - center_pos
    ax.plot(xAux[center_pos - distance_pos:center_pos + distance_pos], S[center_pos - distance_pos:center_pos + distance_pos], '.')
    
test()
test(L=1)

In [ ]:
# def sigmoid(x, a, b, l, s):
#     return a+b*(1/(1+np.power(np.e, -l*(x+s))))

def test(L=None):
    import numpy as np
    import matplotlib.pyplot as plt

    def derivative(x):
        return (b * l) / (np.exp(l * (s + x)) * (1 + np.exp(-l * (s + x)))**2)

    x = [-9.95768915383951, 9.97456435677212]
    OverSampFreq = np.float64(1e3)   
    a, b, l, s, optSmooth = 0.009548118522661594, 0.9765408418125651, 2.883940582009699, 0.004689178501557088, 0.13395440581238172
    if L is not None:
        l = L
    xAux = np.arange(np.min(x), np.max(x), step=1/OverSampFreq)    
    S = mtf_estimator_algorithm.sigmoid(xAux, a, b, l, s)
    fig, ax = plt.subplots(1,1)
    ax.plot(xAux, S, '.')
   
    tax = ax.twinx()
    tax.plot(xAux, derivative(xAux))



    
test()
            




In [ ]:
def _arr_to_scalar(x):
    # If x is a numpy array, return x.item().  This will
    # fail if the array has more than one element.
    return x.item() if isinstance(x, np.ndarray) else x

def old_bound_to_new(bounds):
    """Convert the old bounds representation to the new one.

    The new representation is a tuple (lb, ub) and the old one is a list
    containing n tuples, ith containing lower and upper bound on a ith
    variable.
    If any of the entries in lb/ub are None they are replaced by
    -np.inf/np.inf.
    """

    lb, ub = zip(*bounds)

    # Convert occurrences of None to -inf or inf, and replace occurrences of
    # any numpy array x with x.item(). Then wrap the results in numpy arrays.
    lb = np.array([float(_arr_to_scalar(x)) if x is not None else -np.inf
                   for x in lb])
    ub = np.array([float(_arr_to_scalar(x)) if x is not None else np.inf
                   for x in ub])

    return lb, ub

bounds = [(1e-5, 100.)]
old_bound_to_new(bounds)